In [1]:
import pandas as pd
import csv

def custom_to_csv(data, name):
    data.to_csv(name, index=False)
    f = open(name, "r+")
    lines = f.readlines()
    lines[-1] = lines[-1].rstrip()
    f.close()
    f = open(name, "w+")
    f.writelines(lines)
    f.close()

<h1>Participants</h1>

In [15]:
# Read the FinancialJournal CSV file into a DataFrame
finance_df = pd.read_csv("../data/Datasets/Journals/FinancialJournal.csv")
finance_df

,participantId,timestamp,category,amount
0,0,2022-03-01T00:00:00Z,Wage,2472.507559
1,0,2022-03-01T00:00:00Z,Shelter,-554.988622
2,0,2022-03-01T00:00:00Z,Education,-38.005380
3,1,2022-03-01T00:00:00Z,Wage,2046.562206
4,1,2022-03-01T00:00:00Z,Shelter,-554.988622
...,...,...,...,...
1856325,39,2023-05-25T00:05:00Z,Recreation,-2.760368
1856326,28,2023-05-25T00:05:00Z,Recreation,-29.458409
1856327,370,2023-05-25T00:05:00Z,Recreation,-28.444239
1856328,537,2023-05-25T00:05:00Z,Food,-4.000000


In [16]:
# count participants that dropped out after just a month, we know they are 131
count_fin = finance_df["participantId"].value_counts().reset_index().sort_values("count")
dropped_out = count_fin.head(131)
custom_to_csv(dropped_out["participantId"], "DroppedOut.csv")
dropped_out

,participantId,count
1010,653,11
1009,875,11
1008,346,12
1004,279,14
1007,846,14
...,...,...
882,514,26
884,285,26
885,621,26
881,762,27


In [17]:
# Take the absolute value of the 'amount' column
finance_df["amount"] = finance_df["amount"].abs()
finance_df.head()

,participantId,timestamp,category,amount
0,0,2022-03-01T00:00:00Z,Wage,2472.507559
1,0,2022-03-01T00:00:00Z,Shelter,554.988622
2,0,2022-03-01T00:00:00Z,Education,38.005380
3,1,2022-03-01T00:00:00Z,Wage,2046.562206
4,1,2022-03-01T00:00:00Z,Shelter,554.988622


In [18]:
# Group by 'participantId' and 'category', summing the 'amount' for each group
result_df = finance_df.groupby(["participantId", "category"])["amount"].sum().reset_index()
result_df.head()

,participantId,category,amount
0,0,Education,608.086073
1,0,Food,3868.580758
2,0,Recreation,4870.355051
3,0,Shelter,8879.817947
4,0,Wage,134904.668457


In [21]:
# Write the result to a new CSV file
custom_to_csv(result_df, "AggregatedFinancialJournal.csv")

<hr/>

In [22]:
# read the participants CSV file into a DataFrame
part_df = pd.read_csv("../data/Datasets/Attributes/Participants.csv")
part_df.head()

,participantId,householdSize,haveKids,age,educationLevel,interestGroup,joviality
0,0,3,True,36,HighSchoolOrCollege,H,0.001627
1,1,3,True,25,HighSchoolOrCollege,B,0.328087
2,2,3,True,35,HighSchoolOrCollege,A,0.393470
3,3,3,True,21,HighSchoolOrCollege,I,0.138063
4,4,3,True,43,Bachelors,H,0.857397


In [23]:
total_exp = result_df[(result_df.category != "Wage") & (result_df.category != "RentAdjustment")].groupby("participantId")["amount"].sum()
total_exp

participantId
0       18226.839829
1       21139.514857
2       18363.684618
3       19327.114335
4       33132.259641
            ...     
1006    19239.292195
1007    24756.701369
1008    14491.497800
1009    22273.255338
1010    19331.243416
Name: amount, Length: 1011, dtype: float64

In [24]:
food_exp = result_df[result_df.category == "Food"].groupby("participantId")["amount"].sum()
food_exp

participantId
0       3868.580758
1       3912.751140
2       4265.218707
3       4189.696815
4       4017.282757
           ...     
1006    6559.485799
1007    6644.145592
1008    4306.678867
1009    7924.229957
1010    4284.186971
Name: amount, Length: 1011, dtype: float64

In [25]:
engel_coeff = (food_exp / total_exp).rename("engels")
engel_coeff

participantId
0       0.212246
1       0.185092
2       0.232264
3       0.216778
4       0.121250
          ...   
1006    0.340942
1007    0.268378
1008    0.297187
1009    0.355773
1010    0.221620
Name: engels, Length: 1011, dtype: float64

In [26]:
part_df_augmented = part_df.merge(engel_coeff, on="participantId")
part_df_augmented["haveKids"] = part_df_augmented["haveKids"].map({True: "TRUE", False: "FALSE"})
part_df_augmented

,participantId,householdSize,haveKids,age,educationLevel,interestGroup,joviality,engels
0,0,3,TRUE,36,HighSchoolOrCollege,H,0.001627,0.212246
1,1,3,TRUE,25,HighSchoolOrCollege,B,0.328087,0.185092
2,2,3,TRUE,35,HighSchoolOrCollege,A,0.393470,0.232264
3,3,3,TRUE,21,HighSchoolOrCollege,I,0.138063,0.216778
4,4,3,TRUE,43,Bachelors,H,0.857397,0.121250
...,...,...,...,...,...,...,...,...
1006,1006,1,FALSE,19,HighSchoolOrCollege,J,0.639268,0.340942
1007,1007,1,FALSE,40,HighSchoolOrCollege,B,0.934348,0.268378
1008,1008,1,FALSE,23,Graduate,C,0.163721,0.297187
1009,1009,1,FALSE,39,Low,B,0.828330,0.355773


In [ ]:
custom_to_csv(part_df_augmented, "ParticipantsAugmented.csv")

<hr/>

In [64]:
participants = pd.read_csv("ParticipantsAugmented.csv")
participants

,participantId,householdSize,haveKids,age,educationLevel,interestGroup,joviality,engels
0,0,3,True,36,HighSchoolOrCollege,H,0.001627,0.212246
1,1,3,True,25,HighSchoolOrCollege,B,0.328087,0.185092
2,2,3,True,35,HighSchoolOrCollege,A,0.393470,0.232264
3,3,3,True,21,HighSchoolOrCollege,I,0.138063,0.216778
4,4,3,True,43,Bachelors,H,0.857397,0.121250
...,...,...,...,...,...,...,...,...
1006,1006,1,False,19,HighSchoolOrCollege,J,0.639268,0.340942
1007,1007,1,False,40,HighSchoolOrCollege,B,0.934348,0.268378
1008,1008,1,False,23,Graduate,C,0.163721,0.297187
1009,1009,1,False,39,Low,B,0.828330,0.355773


In [65]:
homes = pd.read_csv("homes.csv")[["participantId", "apartments"]]
homes

,participantId,apartments
0,0,926
1,1,928
2,2,291
3,3,1243
4,4,"194,210,29,375,136"
...,...,...
1006,1006,1633
1007,1007,1620
1008,1008,525
1009,1009,663


In [66]:
def split_apartments(x):
    if type(x) == float:
        return pd.NA # NaN are now pandas.NA
    else:
        return int(x.split(",")[-1])

homes["apartmentId"] = homes["apartments"].apply(split_apartments)
homes.drop("apartments", axis=1, inplace=True)
homes

,participantId,apartmentId
0,0,926
1,1,928
2,2,291
3,3,1243
4,4,136
...,...,...
1006,1006,1633
1007,1007,1620
1008,1008,525
1009,1009,663


In [67]:
apartments = pd.read_csv("ApartmentsCoord.csv")
apartments

,apartmentId,rentalCost,maxOccupancy,numberOfRooms,locationX,locationY,buildingId
0,1,768.16,2,4,1077.697944,648.442716,340
1,2,1014.55,2,1,-185.929284,1520.327098,752
2,3,1057.39,4,3,2123.014186,5126.753457,639
3,4,1259.10,4,3,2103.630178,4266.932930,397
4,5,411.50,1,4,7.058974,79.961637,628
...,...,...,...,...,...,...,...
1512,1729,703.80,2,4,-4097.077997,7409.520667,182
1513,1730,1104.62,4,2,-4334.244917,7164.441147,613
1514,1731,890.69,4,1,-3330.585557,7588.646019,513
1515,1732,703.80,2,3,-4151.807872,7409.520667,182


In [68]:
homesJoined = homes.merge(apartments, on="apartmentId", how="left")
homesJoined.drop(["rentalCost", "maxOccupancy", "numberOfRooms", "buildingId"], axis=1, inplace=True)
homesJoined

,participantId,apartmentId,locationX,locationY
0,0,926,-2721.353174,6862.861219
1,1,928,-1531.132885,5597.244220
2,2,291,-1352.967752,2095.779440
3,3,1243,-1550.046162,5606.726709
4,4,136,1488.478248,3490.893224
5,5,243,-1511.444069,1994.482112
6,6,183,1789.415305,3245.310488
7,7,97,-1025.463021,1571.165821
8,8,321,611.334660,2266.011784
9,9,355,-2025.813803,2039.399099


In [74]:
participantsJoined = participants.merge(homesJoined, on="participantId", how="left")
participantsJoined

,participantId,householdSize,haveKids,age,educationLevel,interestGroup,joviality,engels,apartmentId,locationX,locationY
0,0,3,True,36,HighSchoolOrCollege,H,0.001627,0.212246,926,-2721.353174,6862.861219
1,1,3,True,25,HighSchoolOrCollege,B,0.328087,0.185092,928,-1531.132885,5597.244220
2,2,3,True,35,HighSchoolOrCollege,A,0.393470,0.232264,291,-1352.967752,2095.779440
3,3,3,True,21,HighSchoolOrCollege,I,0.138063,0.216778,1243,-1550.046162,5606.726709
4,4,3,True,43,Bachelors,H,0.857397,0.121250,136,1488.478248,3490.893224
...,...,...,...,...,...,...,...,...,...,...,...
1006,1006,1,False,19,HighSchoolOrCollege,J,0.639268,0.340942,1633,-4227.336286,5302.398117
1007,1007,1,False,40,HighSchoolOrCollege,B,0.934348,0.268378,1620,-3969.050409,6563.139153
1008,1008,1,False,23,Graduate,C,0.163721,0.297187,525,-2158.723147,2525.141681
1009,1009,1,False,39,Low,B,0.828330,0.355773,663,-2366.812535,4880.767220


In [75]:
custom_to_csv(participantsJoined, "ParticipantsAugmented.csv")

<hr/>

In [36]:
aggregatedFinancial = pd.read_csv("AggregatedFinancialJournal.csv")
expensesCategories = aggregatedFinancial["category"].unique()
print(expensesCategories)
aggregatedFinancial

['Education' 'Food' 'Recreation' 'Shelter' 'Wage' 'RentAdjustment']


,participantId,category,amount
0,0,Education,608.086073
1,0,Food,3868.580758
2,0,Recreation,4870.355051
3,0,Shelter,8879.817947
4,0,Wage,134904.668457
...,...,...,...
4137,1009,Wage,28045.271601
4138,1010,Food,4284.186971
4139,1010,Recreation,7601.419125
4140,1010,Shelter,7445.637320


In [44]:
participants = pd.read_csv("ParticipantsAugmented.csv")
for cat in expensesCategories:
    participants[cat] = 0.0

def aptIdtoInt(x):
    if pd.isna(x):
        return pd.NA
    else:
        return int(x)

participants["apartmentId"] = participants["apartmentId"].apply(aptIdtoInt)

In [38]:
def update_expenses(row):
    participants.loc[participants["participantId"] == row["participantId"], [row["category"]]] += row["amount"]

aggregatedFinancial.apply(update_expenses, axis=1)
participants

,participantId,householdSize,haveKids,age,educationLevel,interestGroup,joviality,engels,apartmentId,locationX,locationY,Education,Food,Recreation,Shelter,Wage,RentAdjustment
0,0,3,True,36,HighSchoolOrCollege,H,0.001627,0.212246,926,-2721.353174,6862.861219,608.086073,3868.580758,4870.355051,8879.817947,134904.668457,0.000000
1,1,3,True,25,HighSchoolOrCollege,B,0.328087,0.185092,928,-1531.132885,5597.244220,608.086073,3912.751140,7738.859696,8879.817947,118421.022133,0.000000
2,2,3,True,35,HighSchoolOrCollege,A,0.393470,0.232264,291,-1352.967752,2095.779440,204.999120,4265.218707,4988.619955,8904.846836,104429.003555,0.000000
3,3,3,True,21,HighSchoolOrCollege,I,0.138063,0.216778,1243,-1550.046162,5606.726709,608.086073,4189.696815,5649.513500,8879.817947,100943.911799,0.000000
4,4,3,True,43,Bachelors,H,0.857397,0.121250,136,1488.478248,3490.893224,204.999120,4017.282757,7492.829448,21417.148316,130266.886107,4809.283427
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1006,1006,1,False,19,HighSchoolOrCollege,J,0.639268,0.340942,1633,-4227.336286,5302.398117,0.000000,6559.485799,5124.502128,7555.304268,28160.145453,0.000000
1007,1007,1,False,40,HighSchoolOrCollege,B,0.934348,0.268378,1620,-3969.050409,6563.139153,0.000000,6644.145592,10008.671691,8103.884086,28134.000854,0.000000
1008,1008,1,False,23,Graduate,C,0.163721,0.297187,525,-2158.723147,2525.141681,0.000000,4306.678867,2739.181613,7445.637320,56506.356576,0.000000
1009,1009,1,False,39,Low,B,0.828330,0.355773,663,-2366.812535,4880.767220,0.000000,7924.229957,6903.388062,7445.637320,28045.271601,0.000000


In [43]:
custom_to_csv(participants, "ParticipantsAugmented.csv")

<hr/>

In [48]:
participants = pd.read_csv("ParticipantsAugmented.csv")
participants["apartmentId"] = participants["apartmentId"].apply(aptIdtoInt)

# remove all rows with a NaN value
participantsDropped = participants.dropna()
participantsDropped

,participantId,householdSize,haveKids,age,educationLevel,interestGroup,joviality,engels,apartmentId,locationX,locationY,Education,Food,Recreation,Shelter,Wage,RentAdjustment
0,0,3,True,36,HighSchoolOrCollege,H,0.001627,0.212246,926,-2721.353174,6862.861219,608.086073,3868.580758,4870.355051,8879.817947,134904.668457,0.000000
1,1,3,True,25,HighSchoolOrCollege,B,0.328087,0.185092,928,-1531.132885,5597.244220,608.086073,3912.751140,7738.859696,8879.817947,118421.022133,0.000000
2,2,3,True,35,HighSchoolOrCollege,A,0.393470,0.232264,291,-1352.967752,2095.779440,204.999120,4265.218707,4988.619955,8904.846836,104429.003555,0.000000
3,3,3,True,21,HighSchoolOrCollege,I,0.138063,0.216778,1243,-1550.046162,5606.726709,608.086073,4189.696815,5649.513500,8879.817947,100943.911799,0.000000
4,4,3,True,43,Bachelors,H,0.857397,0.121250,136,1488.478248,3490.893224,204.999120,4017.282757,7492.829448,21417.148316,130266.886107,4809.283427
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1006,1006,1,False,19,HighSchoolOrCollege,J,0.639268,0.340942,1633,-4227.336286,5302.398117,0.000000,6559.485799,5124.502128,7555.304268,28160.145453,0.000000
1007,1007,1,False,40,HighSchoolOrCollege,B,0.934348,0.268378,1620,-3969.050409,6563.139153,0.000000,6644.145592,10008.671691,8103.884086,28134.000854,0.000000
1008,1008,1,False,23,Graduate,C,0.163721,0.297187,525,-2158.723147,2525.141681,0.000000,4306.678867,2739.181613,7445.637320,56506.356576,0.000000
1009,1009,1,False,39,Low,B,0.828330,0.355773,663,-2366.812535,4880.767220,0.000000,7924.229957,6903.388062,7445.637320,28045.271601,0.000000


In [49]:
custom_to_csv(participantsDropped, "ParticipantsAugmented.csv")

<hr/>

<h1>Activities</h1>

In [151]:
# load pubs and restaurants
pubs = pd.read_csv("PubLocation.csv")
pubs["venueType"] = "Pub"
pubs.rename(columns={"pubId": "venueId"}, inplace=True)
pubs.rename(columns={"hourlyCost": "cost"}, inplace=True)
restaurants = pd.read_csv("RestaurantLocation.csv")
restaurants["venueType"] = "Restaurant"
restaurants.rename(columns={"restaurantId": "venueId"}, inplace=True)
restaurants.rename(columns={"foodCost": "cost"}, inplace=True)

# merge pubs and restaurants
activities = pd.concat([pubs, restaurants]).reset_index(drop=True)
activities.drop(["buildingId"], axis=1, inplace=True)
activities

,venueId,cost,maxOccupancy,locationX,locationY,venueType
0,442,8.281103,64,964.438023,3991.603474,Pub
1,443,6.417435,64,1809.880173,4339.172426,Pub
2,444,12.581806,84,770.427904,932.585200,Pub
3,892,11.642905,96,-1524.957321,3815.271490,Pub
4,893,14.840473,79,-1608.766411,3886.492478,Pub
5,894,12.311210,72,-1702.537816,3865.477091,Pub
6,1342,14.140037,77,-260.457470,5026.150804,Pub
7,1343,10.524972,67,-501.426761,3551.572639,Pub
8,1344,10.415293,60,-225.920987,3878.053698,Pub
9,1798,11.545351,64,-3209.738139,6487.657688,Pub


In [152]:
visitsLog = pd.read_csv("VisitsLog.csv")
# create dataframe with venueId and total count of visits
visitsLog = visitsLog.groupby("venueId")["count"].sum().reset_index().rename(columns={"count": "totalVisits"})
activities = activities.merge(visitsLog, on="venueId")
activities

,venueId,cost,maxOccupancy,locationX,locationY,venueType,totalVisits
0,442,8.281103,64,964.438023,3991.603474,Pub,21703
1,443,6.417435,64,1809.880173,4339.172426,Pub,19807
2,444,12.581806,84,770.427904,932.585200,Pub,17592
3,892,11.642905,96,-1524.957321,3815.271490,Pub,23603
4,893,14.840473,79,-1608.766411,3886.492478,Pub,25255
5,894,12.311210,72,-1702.537816,3865.477091,Pub,20253
6,1342,14.140037,77,-260.457470,5026.150804,Pub,50400
7,1343,10.524972,67,-501.426761,3551.572639,Pub,31162
8,1344,10.415293,60,-225.920987,3878.053698,Pub,54174
9,1798,11.545351,64,-3209.738139,6487.657688,Pub,31212


In [153]:
# Load the CSV files
checkin_df = pd.read_csv("../data/Datasets/Journals/CheckinJournal.csv")
financial_df = pd.read_csv("../data/Datasets/Journals/FinancialJournal.csv")
dropped_out = pd.read_csv("DroppedOut.csv")

# Filter CheckIns of dropped out participants
checkin_df = checkin_df[~checkin_df['participantId'].isin(dropped_out["participantId"])]
# Filter Financial Transactions about Food or Recreation
financial_df = financial_df[(financial_df['category'] == 'Food') | (financial_df['category'] == 'Recreation')]
financial_df = financial_df[~financial_df['participantId'].isin(dropped_out["participantId"])]
financial_df["amount"] = financial_df["amount"].abs()

# Convert timestamp columns to datetime dtype
checkin_df['timestamp'] = pd.to_datetime(checkin_df['timestamp'])
financial_df['timestamp'] = pd.to_datetime(financial_df['timestamp'])

# Merge FinancialJournal with CheckInJournal based on participantId
merged_df = pd.merge_asof(financial_df.sort_values('timestamp'), checkin_df.sort_values('timestamp'), 
                          by='participantId', on='timestamp', direction='backward')

# discard rows with NaN values
merged_df = merged_df.dropna()
merged_df = merged_df[(merged_df['venueType'] == 'Pub') | (merged_df['venueType'] == 'Restaurant')]

# drop columns
merged_df = merged_df.drop(columns=["participantId", "category"])
merged_df["venueId"] = merged_df["venueId"].astype(int)

# convert timestamps to only year and month
merged_df['timestamp'] = merged_df['timestamp'].dt.to_period('M')
merged_df

/tmp/ipykernel_152761/401114438.py:30: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  merged_df['timestamp'] = merged_df['timestamp'].dt.to_period('M')


,timestamp,amount,venueId,venueType
16,2022-03,1.025934,894,Pub
50,2022-03,4.438301,895,Restaurant
60,2022-03,4.233449,1348,Restaurant
110,2022-03,14.840473,893,Pub
121,2022-03,4.103737,894,Pub
...,...,...,...,...
1327958,2023-05,2.760368,442,Pub
1327959,2023-05,22.566468,1344,Pub
1327960,2023-05,20.204365,1798,Pub
1327961,2023-05,11.642905,892,Pub


In [154]:
# group each venue and sum its monthly earnings
monthly_earnings = merged_df.groupby(['venueId', 'venueType', 'timestamp']).agg({'amount': 'sum'}).reset_index()
custom_to_csv(monthly_earnings, "EarningsLog.csv")
monthly_earnings

,venueId,venueType,timestamp,amount
0,442,Pub,2022-03,35083.583269
1,442,Pub,2022-04,19956.078227
2,442,Pub,2022-05,16410.385927
3,442,Pub,2022-06,15980.458659
4,442,Pub,2022-07,17177.768145
...,...,...,...,...
475,1805,Restaurant,2023-01,14957.194914
476,1805,Restaurant,2023-02,13491.680678
477,1805,Restaurant,2023-03,15315.183124
478,1805,Restaurant,2023-04,14727.858716


In [157]:
# group each venue and sum its monthly earnings to achieve total earnings
total_earnings = monthly_earnings.groupby(['venueId', 'venueType']).agg({'amount': 'sum'}).reset_index() \
            .rename(columns={"amount": "totalEarnings"}).drop(columns=["venueType"])

# merge total earnings with activities
activities = activities.merge(total_earnings, on="venueId")

,venueId,amount
0,442,262603.439797
1,443,205745.117753
2,444,269969.914434
3,445,43007.279576
4,446,74531.767474
5,447,201316.682435
6,448,134593.913948
7,449,212909.573308
8,892,376873.078889
9,893,479653.969666
